# Causal Inference & Machine Learning: Deep Learning, NLP and Beyond

### Loading Libraries

In [2]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# IterTools
from itertools import combinations

# Statistical Models
from scipy import stats
import statsmodels.api as sm

# Networkx
import networkx as nx

# JavaScript Object Notation
import json

# Data Visualization
import graphviz
import seaborn as sns
import matplotlib.pyplot as plt

# Scikit-Learn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV
from sklearn.metrics import mean_absolute_percentage_error


# DoWhy Causal Libraries
import dowhy
from dowhy import gcm
from dowhy import CausalModel
from dowhy.causal_model import CausalModel

# Notebook Iteration 
from tqdm import tqdm

# Copy
from copy import deepcopy

# Light Gradient Boosting Models
from lightgbm import LGBMRegressor, LGBMClassifier

# PyTorch & Transformers
import torch
from catenets.models.torch import TARNet, SNet, FlexTENet
from models.causal_bert_pytorch.CausalBert import CausalBertWrapper

In [3]:
# Setting Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [5]:
COLORS = [
    '#00B0F0',
    '#FF0000',
    '#B0F000'
]

In [ ]:
def plot_effect(effect_true, effect_pred, figsize=(10, 7), ylim=(5000, 22000)):
    plt.figure(figsize=figsize)
    plt.scatter(effect_true, effect_pred, color=COLORS[0])
    plt.plot(np.sort(effect_true), np.sort(effect_true), color=COLORS[1], alpha=.7, label='Perfect model')
    plt.xlabel('$True\ effect$', fontsize=14, alpha=.5)
    plt.ylabel('$Predicted\ effect$', fontsize=14, alpha=.5)
    plt.ylim(ylim[0], ylim[1])
    plt.legend()
    plt.show()

#### Setting Random Seeds

In [ ]:
SEED = 18

np.random.seed(SEED)
pl.seed_everything(SEED)  

### Going Deeper: Deep Learning for Heterogenous Treatment Effects

#### Data

In [ ]:
SAMPLE_SIZE = 5000
TRAIN_SIZE = 4500
N_FEATURES = 20

X = np.random.normal(0, 1, (SAMPLE_SIZE, N_FEATURES))
T = np.random.binomial(1, 0.5, SAMPLE_SIZE)

weights = np.random.gumbel(5, 10, (SAMPLE_SIZE, N_FEATURES - 1))

y = (50 * T * np.abs(X[:, 0])**1.2) + (weights * X[:, 1:]).sum(axis=1)

y0 = (50 * 0 * np.abs(X[:, 0])**1.2) + (weights * X[:, 1:]).sum(axis=1)
y1 = (50 * 1 * np.abs(X[:, 0])**1.2) + (weights * X[:, 1:]).sum(axis=1)

In [ ]:
effect_true = y1[TRAIN_SIZE:] - y0[TRAIN_SIZE:] 

### S-Learner, X-Learner, DR-Learner & Causal Forest (Benchmark)

In [ ]:
# Placing Benchmarks
benchmark_models = {
    'SLearner': SLearner(overall_model=LGBMRegressor()),
    'XLearner': XLearner(models=LGBMRegressor()),
    'DRLearner': LinearDRLearner(),
    'CausalForest': CausalForestDML()
}

benchmark_results = {}


for model_name, model in benchmark_models.items():
    model.fit(
        X=X[:TRAIN_SIZE, :],
        T=T[:TRAIN_SIZE],
        Y=y[:TRAIN_SIZE]
    )

    effect_pred = model.effect(
        X[TRAIN_SIZE:]
    )
    
    benchmark_results[model_name] = effect_pred

In [ ]:
for model_name in benchmark_results.keys():
    effect_pred = benchmark_results[model_name]
    print(f'{model_name} MAPE = {mean_absolute_percentage_error(effect_true, effect_pred)}')
    
    plot_effect(
        effect_true=effect_true,
        effect_pred=effect_pred,
        ylim=(-150, 250)
    )

### TARNet

In [ ]:
# Setting TarNet
tarnet = TARNet(
    n_unit_in=X.shape[1],
    binary_y=False,
    n_units_out_prop=32,
    n_units_r=8,
    nonlin='selu',
)

# Fitting TarNet
tarnet.fit(
    X=X[:TRAIN_SIZE, :],
    y=y[:TRAIN_SIZE],
    w=T[:TRAIN_SIZE]
);

In [ ]:
# Getting Predictions

effect_pred_tarnet = tarnet.predict(
    X=X[TRAIN_SIZE:, :]
).cpu().detach().numpy()

In [ ]:
# Computing The Error 

mean_absolute_percentage_error(effect_true, effect_pred_tarnet)

In [ ]:
?mean_absolute_percentage_error

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred_tarnet,
    ylim=(-150, 250)
)

### SNet

In [ ]:
# Setting Model
snet = SNet(
    n_unit_in=X.shape[1],
    binary_y=False,
    n_units_out_prop=32,
    n_units_r=8,
    nonlin='selu',
)

# Fitting Model
snet.fit(
    X=X[:TRAIN_SIZE, :],
    y=y[:TRAIN_SIZE],
    w=T[:TRAIN_SIZE]
);

In [ ]:
# SNET Predctions
effect_pred_snet = snet.predict(
    X=X[TRAIN_SIZE:, :]
).cpu().detach().numpy()

In [ ]:
# Computing The Error 

mean_absolute_percentage_error(effect_true, effect_pred_snet)

In [ ]:
plot_effect(
    effect_true=effect_true,
    effect_pred=effect_pred,
    ylim=(-150, 250)
)

### Transformers and Causal Inference

#### Data

In [ ]:
# Retrieving & Reading File

df = pd.read_csv('/Users/isisromero/desktop/causal_inference/data/manga_processed.csv')        

In [ ]:
# Shuffling Observations

df = df.sample(frac=1)

In [ ]:
df.head()

In [ ]:
df.shape

#### Modeling

In [1]:
# Re-running the model training? Un-comment the lines below and collect garbarge to free the GPU memory
# import gc
# del causal_bert
# gc.collect()
# torch.cuda.empty_cache()

# torch.cuda.is_available()

In [ ]:
# Instantiating The Model

causal_bert = CausalBertWrapper(
    batch_size=8,                      
    g_weight=0.05, 
    Q_weight=1., 
    mlm_weight=0.05
)

In [ ]:
# Training
causal_bert.train(
    texts=df['text'], 
    confounds=df['has_photo'], 
    treatments=df['female_avatar'], 
    outcomes=df['upvote'], 
    epochs=6
);

In [ ]:
# Predicting
preds = causal_bert.inference(
    texts=df['text'],
    confounds=df['has_photo'], 
)[0]

In [ ]:
# Computing ATE
np.mean(preds[:, 1] - preds[:, 0])